In [1]:
!tar -xf onnx_quant.tar.xz

In [2]:
!pip install transformers
!pip install torchmetrics
!pip install timm
!pip install onnx
!pip install optimum[exporters]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 27.7 MB

In [3]:
import torch
from transformers import DetrImageProcessor, DetrForObjectDetection
from optimum.onnxruntime import ORTModelForCustomTasks
import numpy as np
import os
import cv2
from tqdm.notebook import tqdm
import time

In [4]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")
qmodel = ORTModelForCustomTasks.from_pretrained('./onnx_quant')

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar

--2023-09-28 11:42:18--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/x-tar]
Saving to: ‘VOCtrainval_11-May-2012.tar’

VOCtrainval_11-May- 100%[===================>]   1.86G  27.9MB/s    in 70s     

2023-09-28 11:43:29 (27.2 MB/s) - ‘VOCtrainval_11-May-2012.tar’ saved [1999639040/1999639040]



In [6]:
!tar -xf ./VOCtrainval_11-May-2012.tar


In [7]:
image_sets_path = './VOCdevkit/VOC2012/ImageSets/Main/'
train_path = os.path.join(image_sets_path, 'train.txt')
val_path = os.path.join(image_sets_path, 'val.txt')

images_path = './VOCdevkit/VOC2012/JPEGImages/'
annots_path = './VOCdevkit/VOC2012/Annotations/'

In [8]:
def read_set(filepath: str):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    return [line[:-1] for line in lines]

In [9]:
val_images = read_set(val_path)
train_images = read_set(train_path)

In [10]:
def read_image_rgb(filepath: str) -> np.array:
    return cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)

In [112]:
from collections import namedtuple
from torchmetrics.detection import MeanAveragePrecision
from bs4 import BeautifulSoup, element

Box = namedtuple('Box', 'xmin ymin xmax ymax')
VOCObject = namedtuple('VOCObject', 'cls xyxy')

def parse_objects(annot):
    return [
        VOCObject(obj.find('name').string, parse_box_xyxy(obj))
        for obj in annot.find_all('object')
    ]

def parse_to_voc_objects(model_res: dict, model_id2lbl: dict):
    """
    model_res: dict results from processor.post_process_object_detection
    """
    labels = [model_id2lbl[lbl_id] for lbl_id in model_res['labels'].cpu().tolist()]
    boxes = [Box(*box.to(int).tolist()) for box in model_res['boxes'].cpu().detach()]
    return [VOCObject(cls, box) for cls, box in zip(labels, boxes)]

coco2voc = {
    'airplane': 'aeroplane',
    'bicycle': 'bicycle',
    'bird': 'bird',
    'boat': 'boat',
    'bottle': 'bottle',
    'bus': 'bus',
    'car': 'car',
    'cat': 'cat',
    'chair': 'chair',
    'cow': 'cow',
    'dining table': 'diningtable',
    'dog': 'dog',
    'horse': 'horse',
    'motorcycle': 'motorbike',
    'person': 'person',
    'potted plant': 'pottedplant',
    'sheep': 'sheep',
    'couch': 'sofa',
    'train': 'train',
    'tv': 'tvmonitor',
}

voc2_id2label = {
    0: 'aeroplane',
    1: 'bicycle',
    2: 'bird',
    3: 'boat',
    4: 'bottle',
    5: 'bus',
    6: 'car',
    7: 'cat',
    8: 'chair',
    9: 'cow',
    10: 'diningtable',
    11: 'dog',
    12: 'horse',
    13: 'motorbike',
    14: 'person',
    15: 'pottedplant',
    16: 'sheep',
    17: 'sofa',
    18: 'train',
    19: 'tvmonitor',
}

voc2_label2id = {lbl: i  for i, lbl in voc2_id2label.items()}

def read_xml(filepath: str):
    with open(filepath, 'r') as xml_file:
        data = xml_file.read()
    return BeautifulSoup(data, 'xml')


def results2pred(results: dict, model_id2lbl: dict) -> dict:
    voc_lbls_mask = torch.tensor([
        model_id2lbl[lbl] in coco2voc.keys()
        for lbl in results['labels'].tolist()
    ])
    if voc_lbls_mask.any():
        return {
            'boxes': results['boxes'][voc_lbls_mask].detach(),
            'scores': results['scores'][voc_lbls_mask].detach(),
            'labels': torch.tensor([
                cocolbl2voc(lbl, model_id2lbl)
                for lbl in results['labels'][voc_lbls_mask].tolist()
            ]),
        }
    else:
        return {
            'boxes': torch.tensor([]).to(float),
            'scores': torch.tensor([]).to(float),
            'labels': torch.tensor([]),
        }

def cocolbl2voc(coco_lbl: int, model_id2lbl: dict) -> int:
    return voc2_label2id[coco2voc[model_id2lbl[coco_lbl]]]

def evaluate_over_voc(images_path, annots_path, val_images, emodel, proc, first):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    map = MeanAveragePrecision(iou_type='bbox')
    eval_time_sec = []
    for img_name in tqdm(val_images, total=len(val_images)):

        img_p = os.path.join(images_path, f'{img_name}.jpg')
        annot_p = os.path.join(annots_path, f'{img_name}.xml')
        val_img = read_image_rgb(img_p)
        val_annot = parse_objects(read_xml(annot_p))

        inputs = proc(images=val_img, return_tensors="pt").to(device)
        st = time.time()
        emodel.to(device)
        outputs = emodel(**inputs)
        outputs['logits'] =  first.class_labels_classifier(outputs['last_hidden_state'])
        outputs['pred_boxes'] = first.bbox_predictor(outputs['last_hidden_state']).sigmoid()
        et = time.time()
        eval_time_sec.append(et - st)
        # target_sizes = torch.tensor([val_img.shape[:-1]])
        target_sizes = torch.tensor([val_img.shape[:-1]])
        results = proc.post_process_object_detection(
            outputs, target_sizes=target_sizes, threshold=0.9
        )[0]
        preds = [results2pred(results, model.config.id2label)]
        targets = [voc_objects2target(val_annot)]
        map.update(preds, targets)

    return map.compute(), round(sum(eval_time_sec) / len(eval_time_sec), 3)

def parse_box_xyxy(annot_obj):
    return Box(
        int(annot_obj.xmin.string),
        int(annot_obj.ymin.string),
        int(annot_obj.xmax.string),
        int(annot_obj.ymax.string),
    )

def voc_objects2target(voc_objects) -> dict:
    gt_boxes, gt_labels = zip(*[
        (list(voc_obj.xyxy), voc2_label2id[voc_obj.cls])
        for voc_obj in voc_objects
    ])
    return {
        'boxes': torch.tensor(gt_boxes).to(float),
        'labels': torch.tensor(gt_labels)
    }


def filter_results(results: dict, model_id2lbl: dict):

    if voc_lbls_mask.any():
        return True, {
            'boxes': results['boxes'][voc_lbls_mask],
            'scores': results['scores'][voc_lbls_mask],
            'labels': results['labels'][voc_lbls_mask],
        }
    else:
        return False, None

In [113]:
results = evaluate_over_voc(images_path=images_path,
                            annots_path=annots_path,
                            val_images=val_images,
                            emodel=model,
                            proc=processor,
                            first = model,
                            )

  0%|          | 0/5823 [00:00<?, ?it/s]

In [114]:
results

({'map': tensor(0.5615),
  'map_50': tensor(0.7287),
  'map_75': tensor(0.6085),
  'map_small': tensor(0.1906),
  'map_medium': tensor(0.4113),
  'map_large': tensor(0.6619),
  'mar_1': tensor(0.4642),
  'mar_10': tensor(0.6254),
  'mar_100': tensor(0.6279),
  'mar_small': tensor(0.2522),
  'mar_medium': tensor(0.4812),
  'mar_large': tensor(0.7130),
  'map_per_class': tensor(-1.),
  'mar_100_per_class': tensor(-1.),
  'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19], dtype=torch.int32)},
 0.08)

In [115]:
1000 / 80

12.5